<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-1-Introduction-to-Generative-AI-and-Agentic-AI/M1LC6_Implementing_Tools_%26_Tool_Calling_for_Agentic_AI_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Tools & Tool Calling for Agentic AI Systems

## Tools for Agentic AI Systems

In Agentic AI systems, tools are functions that allow the large language model (LLM) to do things it cannot do on its own. These include actions like searching for information, looking up data from a database, performing calculations, or calling APIs.



## Tool Calling for Agentic AI Systems

Tool calling is what allows the LLM to request a tool when it needs help answering a question. In LangChain, this is done by using the `bind_tools` method. This tells the model what tools are available, how they work, and when to use them when we pass input queries to the LLM.

We can also leverage LangGraph and create react agents by passing tools, prompts and LLMs to built-in functions, something we will look at in future demos

Remember the LLM will not call the tools for us, we will have to either execute them manually or as we will see in the future live demos, our AI Agents will execute them for us.


## Problem Statement

We will be building an Agentic AI system - HealthBuddy, which leverages:

- ReAct principles (Reasoning + Acting)
- LLM + tools + instructions
- Access external information sources (PubMed and Web Search tools)
to provide helpful, actionable answers to user health queries — including symptom checking, treatment suggestions etc.
- Provide appropriate doctor recommendations (using a doctor recommendation tool)

The focus in this notebook is to understand how to build custom tools for Agentic AI Systems and connect them to a regular Large Language Model (LLM) to enable it to be able to request tool calls when it needs access to more information to be able to give a better answer to the user query


---

### Objective:

We want to:

- Understand how to design and register custom tools
- Enable LLMs to dynamically invoke tools via function/tool calling

This notebook focuses on building tools and demonstrating tool-calling as depicted in the following workflow.

![](https://i.imgur.com/11KoTnb.png)


In the next notebook we will reuse this to build our first Agentic AI System

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 arxiv==2.2.0 pymupdf==1.26.3 --quiet

## Load Necessary Dependencies

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

## Setup Authentication

In [ ]:
import os
import getpass

# OpenAI API Key (for chat & embeddings)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (https://platform.openai.com/account/api-keys):\n")

# Tavily API Key (for web search)
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key (https://app.tavily.com/home):\n")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Preparing Database for Doctor Recommendations Tool

In this section, we will create a small, in-memory database that contains information about doctors. This data will be used by our **Doctor Recommendation Tool**, which will help users find the right doctor based on their health query or symptoms.

The database includes a list of doctors along with their:
- Name
- Specialization (e.g., Dermatology, Pediatrics, Cardiology)
- Location
- Availability
- Contact information

We are using a simple Python list of dictionaries to store the doctor data. This keeps it easy to understand and modify. In a real-world application, this would typically be replaced by a backend database like PostgreSQL, MongoDB, or an external API.

We will build a tool later on to use this data to recommend doctors based on the user's needs — for example, suggesting a pediatrician for a child’s fever or a cardiologist for chest pain.

In [ ]:
# loading our doctors dataset
doctors_db = [
    {"name": "Dr. Janet Dyne", "specialization": "Endocrinology (Diabetes Care)", "available_timings": "10:00 AM - 1:00 PM", "location": "City Health Clinic", "contact": "janet.dyne@healthclinic.com"},
    {"name": "Dr. Don Blake ", "specialization": "Cardiology (Heart Specialist)", "available_timings": "2:00 PM - 5:00 PM", "location": "Metro Cardiac Center", "contact": "don.blake@metrocardiac.com"},
    {"name": "Dr. Susan D'Souza", "specialization": "Oncology (Cancer Care)", "available_timings": "11:00 AM - 2:00 PM", "location": "Hope Cancer Institute", "contact": "susan.dsouza@hopecancer.org"},
    {"name": "Dr. Matt Murdock", "specialization": "Psychiatry (Mental Health)", "available_timings": "4:00 PM - 7:00 PM", "location": "Mind Care Center", "contact": "matt.murdock@mindcare.com"},
    {"name": "Dr. Dinah Lance", "specialization": "General Physician", "available_timings": "9:00 AM - 12:00 PM", "location": "Downtown Medical Center", "contact": "dinah.lance@downtownmed.com"}
]

## Create Tools for AI Agent

In this section, we will define the tools that our AI Agent will use to perform specific tasks.

LangChain makes it easy to create and register tools using the `Tool` class. A tool includes:
- A name and description
- The python function to be called
- An input schema that tells the model what arguments it can use

When tools are defined properly, they help the model solve more complex problems by letting it take actions and use external data. This makes the system more useful and reliable.

### 🧪 Example
```python
from langchain.tools import tool

@tool
def search_web(query: str) -> str:
    """Get live information for user queries from the web."""
    # assuming we have a google_search function implemented to search on google
    return google_search(query)
```

These tools will allow the agent to retrieve information from the external environment, as well as recommend doctors from our in-memory doctor database.

The goal is to modularize the logic for different types of tasks into reusable components that can be invoked by the LLM when needed. These include:

- A **Web Search Tool** that queries the web to simulate general online web search
- A **arXiv Search Tool** that retrieves information from arXiv for research-grade medical content
- A **Doctor Recommendation Tool** that finds suitable doctors based on user symptoms or needs

This tool-based setup is essential for enabling agentic behavior, where the LLM reasons through a problem, decides which action to take, and requests to call the right tools to gather more information or perform a task.


In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.retrievers import ArxivRetriever

# Tool for web search on general health topics
# Tavily Web Search
tavily_search = TavilySearchAPIWrapper()

@tool
def search_web(query: str) -> list:
    """Search the web for a query. Useful for getting general information or upto date information about healthcare topics."""
    # Perform web search on the internet
    results = tavily_search.raw_results(query=query, max_results=5, search_depth='advanced',
                                        include_answer=False, include_raw_content=True)
    docs = results['results']
    docs = [doc for doc in docs if doc.get("raw_content") is not None]
    docs = ['## Title\n'+doc['title']+'\n\n'+'## Content\n'+doc['raw_content']+'\n\n'+'##Source\n'+doc['url'] for doc in docs]
    return docs


# Tool for arXiv Search
# arxiv search retriever
arxiv_retriever = ArxivRetriever(
    top_k_results=3,
    get_full_documents=True,
    doc_content_chars_max=20000
)
@tool
def search_arxiv(query: str) -> list:
    """Search arXiv for scientific articles related to the query."""
    try:
        results = arxiv_retriever.invoke(query)
        if results:
            articles = ['## Title\n'+doc.metadata['Title']+'\n\n'+'## Summary\n'+doc.metadata['Summary']+'\n\n'+'##Content\n'+doc.page_content for doc in results]
            return articles
        else:
            return ["No articles found for the given query."]
    except Exception as e:
        return [f"Error fetching arXiv articles: {str(e)}"]


# Tool for recommending a doctor based on user symptoms or query
@tool
def recommend_doctor(query: str) -> dict:
    """Recommend the most suitable doctor based on the user's symptoms."""
    doctors_list = str(doctors_db)
    # Use LLM reasoning to choose the most appropriate doctor based on the query
    prompt = f"""You are an assistant helping recommend a doctor based on patient's health issues.

                 Here is the list of available doctors:
                {doctors_list}

                Given the user's query: "{query}"

                Choose the most suitable doctor from the list. Only pick one doctor.
                Return only the selected doctor's information in JSON format (no markdown).
                If not sure, recommend the General Physician.
              """

    response = llm.invoke(prompt)
    return {"recommended_doctor": response.content}

## Testing out our Tools

You can just call the above tools manually by using the `.invoke(...)` function with the necessary inputs.

The following code shows some example queries and outputs on calling these tools

In [ ]:
results = search_web.invoke('Popular treatments in Diabetes')
print(len(results))
print(results[0][:3000])

In [ ]:
results = search_arxiv.invoke('treatments in Diabetes')
print(len(results))
print(results[0][:3000])

In [ ]:
result = recommend_doctor.invoke('Treatments for Diabetes')
print(result)

## Explore LLM tool calling with custom tools

An agent is basically an LLM which has the capability to automatically call relevant functions to perform complex or tool-based tasks based on input human prompts.

Tool calling also popularly known as function calling is the ability to reliably enable such LLMs to call external tools and APIs.

We will leverate the custom tools we created earlier in the previous section and try to see if the LLM can automatically call the right tools based on input prompts

## Tool calling for LLMs

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! The model is coming up with the arguments to a tool, and actually running the tool (or not) is up to the user or agent defined by the user.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools.

For example, if a user asks a question that requires a search or data lookup, the model can decide to call a specific tool with a tool call request.

Tool calling in LangChain works by:
1. Registering defined tool functions using `@tool` decorator
2. Binding the tools to the model using `llm.bind_tools([tool1, tool2, ...])`
3. Passing a user query to the bound model
4. Letting the model decide whether to use a tool or respond directly

This setup lets the model behave more like an agent that can take actions, observe results, and continue the conversation intelligently.

By using `bind_tools`, we give the LLM the ability to understand what tools are available and make requests to use them only when needed.

### 🧪 Example
```python
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([search_web])

response = llm_with_tools.invoke("treatments for diabetes?")
```

### Tool Call Requests

LLMs will not call and execute the tools for you directly but will request tool calls based on reasoning on the input user query if they feel that they do not have enough information to answer the question directly.

To experiment with this, let's first define our tools and bind them to our LLM

In [ ]:
# List of all tools that the LLM should be aware of
# These tools were defined earlier using the @tool decorator
tools = [search_web, search_arxiv, recommend_doctor]

# Bind the tools to the LLM so it can request tool calls when needed
# This enables tool calling functionality in LangChain
llm_with_tools = llm.bind_tools(tools=tools)

Now we can test out these prompts using out LLM with tools

In [ ]:
prompts = [
    "treatments available for diabetes",
    "Research papers on diabetes treatments",
    "What doctor could I visit for diabetes",
    "Explain what is diabetes in simple terms"
]

results = []

for prompt in prompts:
    result = llm_with_tools.invoke(prompt)
    results.append(result)

Observe that:

- If the LLM can answer the question directly, the response will be present in `result.content`
- If the LLM can't answer the question directly, the response will be empty but will contain one or more tool call requests in `result.tool_calls`

In [ ]:
for result in results:
    # If the model gave a direct response without needing a tool
    if result.content:
        print('No tool call was needed')
        print('Direct LLM Response:', result.content)

    # If the model decided that a tool should be called
    if result.tool_calls:
        print('LLM needs to call tools')
        print(result.tool_calls)

    print()

### Tool Call Responses with Manual Tool Calling

While an Agentic Framework like LangChain or LangGraph can leverage AI Agents themselves to handle tool call requests by calling them automatically and getting tool call results (to further feed it to the LLM for more reasoning), here we will call these tool call requests manually just to show how the results would look like.

In future demos the AI Agents will handle this automatically.

We start off by creating a mapping between the tool call names (strings) and the actual tool functions (python function names)

In [ ]:
# create a mapping between tool call request name and the actual tool call function
tool_mapper = {
    "search_web": search_web,
    "search_arxiv": search_arxiv,
    "recommend_doctor": recommend_doctor
}

If you inspect the tool call requests for any input prompt,
we have the tool name and the input arguments which will need to be extracted and then we will need to call the actual tool with the input arguments

For example based on the following tool call request, we would need to call:

`search_web.invoke(query='treatments available for diabetes')`

In [ ]:
# check out a sample tool call request
results[0].tool_calls

We can easily automate this by extracting the tool call request `name` and `args` and invoking the actual python tool function matching the `name` with the input arguments (`args`)

In [ ]:
for result in results:
    tool_call_requests = result.tool_calls
    for tool_call in tool_call_requests:
        # Get the actual tool function from the name using the mapping
        selected_tool = tool_mapper[tool_call["name"]]

        # Execute the tool with the provided arguments from the LLM
        print(f"Calling tool: {tool_call['name']}")
        tool_output = selected_tool.invoke(tool_call["args"])
        print(tool_output)
        print()


In the upcoming hands-on demo you will see how to connect LLMs, tools and instruction prompts together to automate this and build your first Agentic AI System with Tool-Use!